## **Simulando o Feixe**

Esse trabalho tem como objetivo simular o comportamento de um feixe de elétrons em acelerador linear, bem como a sua correção utilizando eletroímãs de quadrupolo.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Slider

# %matplotlib widget

plt.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "font.size": 12 
})

Equações principais:

$$
\vec{F} = m \cdot \vec{a} = \frac{d\vec{p}}{dt}
$$

$$
\gamma = \sqrt{1 + \left(\frac{|\vec{v}|}{c} \right)^2} = \sqrt{1 + \left(\frac{|\vec{p}|}{m_0 c}\right)^2}
$$

$$
\vec{v} = \frac{\vec{v}}{\gamma} = \frac{\vec{p}}{m_0 \gamma} \approx c
$$

<!-- $$
\vec{v} = v_x \hat{i} + v_y \hat{j} + v_z \hat{k} \rightarrow |\vec{v}| = \sqrt{v_x^2 + v_y^2 + v_z^2} \approx c
$$ -->

$$
\vec{B} = B_x \hat{i} + B_y \hat{j} + B_z \hat{k}
$$

$$
\vec{F}^{N+1} = \vec{F}^{N} + \vec{f}_{ext} \cdot \Delta t = \vec{F}^{N} + q \cdot \vec{v} \times \vec{B} \cdot \Delta t
$$

In [6]:
config = {
    "V_LUZ": 2.99792458e8, # m\s
    "MASSA_E": 9.1093837e-31, # kg
    "CARGA_E": 1.60217663e-19, # C
    "HEIGHT": 100,
    "WIDTH": 100,
}

parametros = {
    "vx0": 0.0,
    "vy0": 0.0,
    "vz0": 1e-8 * config["V_LUZ"],
    "x0": config["WIDTH"] / 2,
    "y0": config["HEIGHT"] / 2,
    "z0": 0.0,
    "forca": [0, 0, 1e-16],
    "dt": 1e-11,
}       

In [ ]:
class FeixeEletron():
    def __init__(self, parametros, config):
        self.c = config["V_LUZ"]
        self.m_e = config["MASSA_E"]
        self.dt = parametros["dt"]

        self.forca = np.array(parametros["forca"], dtype=float)
        self.pos0 = np.array([parametros["x0"], parametros["y0"], parametros["z0"]], dtype=float)
        self.vel0 = np.array([parametros["vx0"], parametros["vy0"], parametros["vz0"]], dtype=float)

        v0_mag = np.linalg.norm(self.vel0)
        self.gamma0 = 1.0 / np.sqrt(1.0 - (v0_mag/self.c)**2)
        self.momento0 = self.m_e * self.vel0 * self.gamma0

    def momento_para_velocidade(self, momento):
        momento_norm = np.linalg.norm(momento)
        gamma = 1 / np.sqrt(1 + (momento_norm)/(self.m_e * self.c))

        velocidade = momento / (gamma * self.m_e)

        return velocidade
    
    def runge_kutta(self, n_times):
        self.pos = np.zeros((n_times + 1, 3))
        self.vel = np.zeros((n_times + 1, 3))
        self.momentos = np.zeros((n_times + 1, 3))
        self.t = np.zeros(n_times + 1)

        self.pos[0] = self.pos0
        self.vel[0] = self.vel0

        momento_atual = self.momento0
        pos_atual = self.pos0
    
        for i in range(n_times):
            # k1
            momento_k1 = momento_atual
            v_k1 = self.momento_para_velocidade(momento_k1)       
            forca_k1 = self.forca

            # k2
            momento_k2 = momento_atual + 0.5 * self.dt * forca_k1 
            v_k2 = self.momento_para_velocidade(momento_k2)
            forca_k2 = self.forca

            # k3
            momento_k3 = momento_atual + 0.5 * self.dt * forca_k1
            v_k3 = self.momento_para_velocidade(momento_k3)
            forca_k3 = self.forca

            # k4
            momento_k4 = momento_atual + self.dt * forca_k3
            v_k4 = self.momento_para_velocidade(momento_k4)
            forca_k4 = self.forca

            # somando
            momento_atual += (self.dt / 6) * (forca_k1 + 2*forca_k2 + 2*forca_k3 + forca_k4)
            pos_atual += (self.dt / 6) * (v_k1 + 2*v_k2 + 2*v_k3 + v_k4)
            
            self.momentos[i+1] = momento_atual
            self.vel[i+1] = self.momento_para_velocidade(momento_atual)
            self.pos[i+1] = pos_atual
            self.t[i+1] = (i+1) * self.dt  

N_VEZES = 50000

eletron = FeixeEletron(parametros, config)
eletron.runge_kutta(N_VEZES)
eletron.pos[0:10]
